## PAIR PROGRAMMING ETL III

### ETL Transformación II - Clases y Funciones de limpieza
---

En la lección de hoy aprendimos como Crearnos una clase que nos permita limpiar los datos obtenidos de la API.

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import ast
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" 
pd.options.display.max_columns = None

In [2]:
class Ataquesclima:

    def __init__(self, dicc_datos):

        self.dicc_datos= dicc_datos

    def conexion_API(self, producto):
        
        # Especificamos el producto del que queremos extraer la información
        self.producto= producto
       
        # Creamos un dataframe vacío con las columnas que rellenaremos con la información obtenida de la API
        df_clima_paises = pd.DataFrame()

        for key, value in self.dicc_datos.items(): 

            url =f'http://www.7timer.info/bin/api.pl?lon=-{self.dicc_datos[key][0]}&lat={self.dicc_datos[key][1]}&product={self.producto}&output=json'

            response = requests.get(url=url)
            codigo_estado = response.status_code
            razon_estado = response.reason
            if codigo_estado == 200:
                print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
            elif codigo_estado == 402:
                print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
            elif codigo_estado == 404:
                print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
            else:
                print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)

            # Convetimos la información obtenida en formato json a un dataframe
            df = pd.json_normalize(response.json()['dataseries']) 
            # Creamos una nueva columna para que inserte la key que corresponde al nombre del país
            df['country']=key
            # Concatenamos los dataframes de cada país en uno
            df_clima_paises=pd.concat([df_clima_paises,df], axis=0, ignore_index = True)
        return df_clima_paises


    def limpiar_meteo(self, df_clima_paises): 

        self.df_clima_paises = df_clima_paises
        
        #df_clima_paises['rh_profile']= df_clima_paises['rh_profile'].apply(ast.literal_eval)
        # Para separar la lista de diccionarios en varias columnas
        x = df_clima_paises['rh_profile'].apply(pd.Series)     

        # For loop para sacar el nombre de la columna y los valores de las filas
        for i in range(len(x.columns)): 
        
            # aplicamos el apply,extraemos el valor de la key "layer" y lo almacenamos en una variable que convertimos a string 
            nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 
            # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
            valores = list(x[i].apply(pd.Series)["rh"] )

            # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df_clima_paises.insert(i, nombre, valores)
        pass
            
        #df_clima_paises['wind_profile']= df_clima_paises['wind_profile'].apply(ast.literal_eval)
        y = df_clima_paises['wind_profile'].apply(pd.Series)

        # For loop para sacar el nombre de la columna y los valores de las filas
        for i in range(len(y.columns)): 
                
            # aplicamos el apply,extraemos los valores de la key "layer" y lo almacenamos en dos variables que convertimos a strings
            nombre1 = "direction" + str(y[i].apply(pd.Series)["layer"][0]) 
            nombre2 = "speed" + str(y[i].apply(pd.Series)["layer"][0]) 

            # hacemos lo mismo con dos variables para "guardar" los valores
            valores1= list(y[i].apply(pd.Series)["direction"] )
            valores2= list(y[i].apply(pd.Series)["speed"] )

            # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df_clima_paises.insert(i, nombre1, valores1)
            df_clima_paises.insert(i,nombre2,valores2)
        pass

        # Eliminamos las columnas que tienen las listas de diccionarios, información duplicada
        df_clima_paises.drop(['rh_profile','wind_profile'], axis=1, inplace=True)
         

        #Calculamos la media por pais
        df_clima_paises = df_clima_paises.groupby('country').mean()
            

        #Calculamos la fecha del dato y añadimos la columna al dataframe
        hoy = datetime.now()
        hoy = datetime.strftime(hoy, '%Y-%m-%d')
        df_clima_paises["fecha"] = hoy
        

        #reseteamos el indice para guardar correctamente el df.
        df_clima_paises.reset_index(inplace=True)
        

        print(f'Ya está listo el dataframes de meteo para juntarlo con el de ataques')
        return df_clima_paises
            

    def juntar_dfs(self, df_attacks, df_clima_paises): 
        
        self.df_attacks = df_attacks
        self.df_clima_paises = df_clima_paises

        df_union= df_attacks.merge(df_clima_paises, how= 'inner', on= 'country')


        print("El df de union tiene las siguientes columnas: \n", list(df_union.columns))
      
        # guardamos los datos
        df_union.to_pickle('../files/datos_clima_attacks_clase.pkl')
      
        return df_union

    def chequear_datos(self, df_union): 

        print(f"El número de filas son: {df_union.shape[0]}\ny el número de columnas son: {df_union.shape[1]}")
        print("-----------------------------------------")

        print("Los tipos de datos que tenemos son:", "\n")
        print(df_union.dtypes)
        print("-----------------------------------------")

        print("El porcentaje de nulos:", "\n")
        print((df_union.isnull().sum() / df_union.shape[0]) *  100)


In [3]:
# Hacemos un diccionario con los países como keys y las coordenadas como values
dicc_datos= {'usa': [-100.445882, 39.7837304],'australia': [134.755, -24.7761086],'south africa': [24.991639, -28.8166236],'new zealand': [172.8344077, -41.5000831],'papua new guinea': [144.2489081, -5.6816069]}

In [4]:
api = Ataquesclima(dicc_datos) # Iniciamos la clase

In [5]:
df_clima_paises= api.conexion_API('meteo') # Hacemos la llamada a la API a traves del metodo creado en la clase.
df_clima_paises.head(2)

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 235, 'speed':...",13,15,3,1026,none,0,0,195,3,usa
1,6,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 220, 'speed':...",14,10,9,1026,none,0,0,215,3,usa


In [6]:
df_clima_paises.shape

(320, 17)

In [7]:
df_clima_paises = api.limpiar_meteo(df_clima_paises)# Aplicamos el metodo de limpiar creado en la clase.


Ya está listo el dataframes de meteo para juntarlo con el de ataques


/var/folders/db/hp0l5_js4yv8br5y2tc3frpm0000gn/T/ipykernel_13057/2913998441.py:84: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_clima_paises = df_clima_paises.groupby('country').mean()


In [8]:
df_clima_paises.head()

,country,speed950mb,speed900mb,speed850mb,speed800mb,speed750mb,speed700mb,speed650mb,speed600mb,speed550mb,speed500mb,speed450mb,speed400mb,speed350mb,speed300mb,speed250mb,speed200mb,direction200mb,direction250mb,direction300mb,direction350mb,direction400mb,direction450mb,direction500mb,direction550mb,direction600mb,direction650mb,direction700mb,direction750mb,direction800mb,direction850mb,direction900mb,direction950mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.speed,fecha
0,australia,3.375000,3.343750,3.203125,3.187500,3.078125,3.062500,3.125000,3.31250,3.500000,3.562500,3.578125,3.562500,3.593750,3.828125,4.265625,5.046875,137.343750,121.796875,121.875000,127.187500,125.000000,103.593750,101.718750,94.296875,101.484375,84.53125,81.171875,73.046875,65.781250,64.921875,67.265625,72.812500,13.421875,11.500000,8.109375,5.593750,4.140625,3.546875,3.171875,3.265625,3.046875,3.187500,3.562500,3.359375,2.531250,2.171875,4.265625,5.828125,97.5,3.015625,-9999.0,-9999.0,-9999.0,25.828125,-2.781250,10.359375,1016.171875,2.546875,0.0,3.171875,2023-01-12
1,new zealand,3.937500,3.781250,3.703125,3.750000,3.640625,3.640625,3.703125,3.68750,3.734375,3.859375,3.968750,4.265625,5.015625,5.296875,6.062500,6.109375,231.484375,187.578125,196.328125,170.156250,162.968750,146.015625,145.468750,137.656250,132.421875,131.09375,126.250000,129.531250,126.328125,129.140625,113.906250,116.328125,13.125000,13.437500,6.218750,3.765625,3.421875,2.921875,2.062500,1.343750,0.937500,1.468750,2.453125,4.812500,5.406250,6.421875,6.953125,2.718750,97.5,7.187500,-9999.0,-9999.0,-9999.0,15.062500,10.187500,10.468750,1019.281250,2.187500,0.0,3.546875,2023-01-12
2,papua new guinea,4.046875,4.468750,4.593750,4.453125,3.906250,3.234375,2.281250,2.59375,3.453125,4.437500,5.000000,4.937500,5.187500,5.140625,5.796875,6.281250,235.000000,228.437500,244.296875,254.453125,256.718750,260.156250,253.437500,240.156250,217.421875,150.15625,78.906250,81.328125,83.359375,83.281250,81.796875,81.640625,13.750000,11.296875,7.906250,4.046875,1.265625,-0.312500,0.093750,2.296875,2.562500,2.062500,2.343750,4.906250,5.765625,5.718750,6.015625,2.437500,97.5,4.250000,-9999.0,-9999.0,-9999.0,25.703125,-0.437500,11.265625,1009.937500,2.359375,0.0,3.250000,2023-01-12
3,south africa,2.578125,2.343750,2.234375,2.265625,2.234375,2.359375,2.406250,2.43750,2.609375,2.968750,3.453125,3.765625,3.875000,3.906250,4.156250,4.500000,214.296875,228.359375,229.609375,230.390625,228.281250,224.218750,222.890625,221.484375,217.109375,221.25000,219.609375,207.500000,180.625000,166.015625,138.203125,119.453125,13.140625,10.546875,8.281250,8.046875,9.359375,9.750000,10.140625,9.140625,6.218750,5.046875,2.656250,1.359375,2.437500,4.156250,5.625000,7.328125,97.5,5.390625,-9999.0,-9999.0,-9999.0,23.687500,2.093750,10.609375,1019.343750,1.421875,0.0,2.531250,2023-01-12
4,usa,3.328125,3.734375,3.984375,4.156250,4.265625,4.500000,4.609375,4.96875,5.312500,6.000000,6.593750,7.015625,7.484375,8.218750,8.765625,9.187500,288.437500,281.640625,274.296875,289.140625,289.453125,293.828125,296.718750,286.640625,292.343750,282.96875,275.078125,283.828125,296.250000,281.718750,258.437500,244.062500,3.656250,3.937500,3.781250,3.109375,2.312500,2.171875,2.609375,2.953125,3.640625,4.187500,5.109375,4.390625,3.671875,4.656250,5.015625,4.187500,97.5,4.062500,-9999.0,-9999.0,-9999.0,12.203125,9.859375,3.671875,1013.640625,0.609375,0.0,2.953125,2023-01-12


In [9]:
df_clima_paises.shape # Verificamos la cantidad de columnas del df de clima.

(5, 62)

In [10]:
df_attacks = pd.read_pickle("../files/attacks12_remplazo_nulos.pkl") #Abrimos el archivo de ataques.

In [11]:
df_attacks.shape #Verificamos la cantidad de columnas del df attacks.

(1672, 10)

In [12]:
df_union= api.juntar_dfs(df_attacks, df_clima_paises)# Unimos el dataframe de clima y el de ataques.

El df de union tiene las siguientes columnas: 
 ['case_number', 'year', 'mes', 'sexo', 'edades', 'country', 'type', 'activity', 'fatal', 'cat_species', 'speed950mb', 'speed900mb', 'speed850mb', 'speed800mb', 'speed750mb', 'speed700mb', 'speed650mb', 'speed600mb', 'speed550mb', 'speed500mb', 'speed450mb', 'speed400mb', 'speed350mb', 'speed300mb', 'speed250mb', 'speed200mb', 'direction200mb', 'direction250mb', 'direction300mb', 'direction350mb', 'direction400mb', 'direction450mb', 'direction500mb', 'direction550mb', 'direction600mb', 'direction650mb', 'direction700mb', 'direction750mb', 'direction800mb', 'direction850mb', 'direction900mb', 'direction950mb', 'rh_950mb', 'rh_900mb', 'rh_850mb', 'rh_800mb', 'rh_750mb', 'rh_700mb', 'rh_650mb', 'rh_600mb', 'rh_550mb', 'rh_500mb', 'rh_450mb', 'rh_400mb', 'rh_350mb', 'rh_300mb', 'rh_250mb', 'rh_200mb', 'timepoint', 'cloudcover', 'highcloud', 'midcloud', 'lowcloud', 'temp2m', 'lifted_index', 'rh2m', 'msl_pressure', 'prec_amount', 'snow_depth', '

In [13]:
df_union.shape #Chequeamos que la union se realizo bien.

(1355, 71)

In [14]:
api.chequear_datos(df_union)# Llamamos al metodo de chequear datos.

El número de filas son: 1355
y el número de columnas son: 71
-----------------------------------------
Los tipos de datos que tenemos son: 

case_number       object
year             float64
mes               object
sexo              object
edades           float64
                  ...   
msl_pressure     float64
prec_amount      float64
snow_depth       float64
wind10m.speed    float64
fecha             object
Length: 71, dtype: object
-----------------------------------------
El porcentaje de nulos: 

case_number      0.000000
year             0.000000
mes              4.723247
sexo             0.000000
edades           0.000000
                   ...   
msl_pressure     0.000000
prec_amount      0.000000
snow_depth       0.000000
wind10m.speed    0.000000
fecha            0.000000
Length: 71, dtype: float64
